In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from math import isnan
import sys
import os
from sklearn.model_selection import train_test_split
from config import Config
from equation import get_equation
from solver import dBSDE
import matplotlib.pyplot as plt
%matplotlib inline

# Dyanmic portfolio optimization

The investor is facing a optimization problem that she wants to maiximize her expected utility over the investment horizon given the initial wealth and the access to a financial market consists of a riskless asset and some risky assets. The opitmization problem can be formulated as follows
\begin{align}
V_{0} & =\sup_{({\pi},c)\in\mathcal{A}} \mathbb{E}[\int_{0}^{T}f(c_{s,}V_{s}^{(\pi,c)})ds+U(T,\mathcal{W}_{T}^{({\pi},c)})]\nonumber \\
\text{s.t. }d{S}_{t} &=\text{diag}({S}_{t})\big[\big(r(t,{X}_{t})-p(t,X_t)\big)\mathbf{1}_{d}dt+{\sigma}(t,{X}_{t})\big({{\theta}}(t,{X}_{t})dt+d{W}_{t}\big)\big]\\
d\mathcal{W}_{t}^{({\pi},c)} & =\mathcal{W}_{t}^{({\pi},c)}[r_{t}(X_{t})dt+{\pi}_{t}^{\intercal}{{\sigma}}_{t}({X}_{t})({{\theta}}_{t}({X}_{t})dt+d{W}_{t})]-c_{t}dt\label{eq:3}\\
d{X}_{t} & =\mathbf{{\alpha}}(t,{X}_{t})dt+{{\beta}}(t,{X}_{t})d{W}_{t}
\end{align}

## Recursive utility

Recursive utility is also known as the continuous-time Epstein-Zin utility. The aggregator function $f$ is given by

$$
f(c,v):=\delta\vartheta v[(\frac{c}{((1-\gamma)v)^{\frac{1}{1-\gamma}}})^{1-\frac{1}{\psi}}-1]
$$
where

- $\delta$: rate of time preference
- $\gamma$: relative risk aversion
- $\psi$:EIS (elasticity of intertemporal substitution)
- $\vartheta:=\frac{1-\gamma}{1-\phi}$, $\phi:=1/\psi$

We consider the bequest utility $U(T,c)=e^{-\delta T}\frac{c^{1-\gamma}}{1-\gamma}$

The standard time-additive power utility is a special case of the recursive utility.

Note that when $\gamma=1/\psi$, Epstein-Zin utility reduces to power utility, in which case 
$$
V_0=\mathbb{E}[\int_{0}^{T}e^{-s\beta}u(c_t)ds+e^{-\delta T}u(\mathcal{W}_T^{(\pi,c)})]
$$
and
$$
f=\delta\big(\frac{c^{1-\gamma}}{1-\gamma}\big)-\delta v
$$

## Optimal consumption-portfolio plan
By the homothetic property of Epstein–Zin utility, we speculate the utility evaluated at the optimal strategy has the decomposition $V_{t}^{*}=\frac{\mathcal{W}_{t}^{1-\gamma}}{1-\gamma}e^{Y_{t}},\;t\in[0,T]$ where Y satisfies the BSDE $$Y_{t}=\eta + \int_{t}^{T}g(s,Y_{s},{Z}_{s})ds-\int_{t}^{T}{Z}_{s}d{W}_{s}$$

The optimal consumption-portfolio plan is determined by $Y$ and $Z$
\begin{align*}
c_t^{*}/\mathcal{W_t} & =\delta^{\psi}e^{-\frac{\psi}{\vartheta}Y_t}\\
{\pi_t}^{*} & =\frac{1}{\gamma}{\sigma_t}^{-1}{\theta_t}+\frac{1}{\gamma}{\sigma_t}^{-1}{Z_t}
\end{align*}

Therefore, our goal is to solve the following FBSDEs
\begin{align*}
Y_{t} &=\eta+\int_{t}^{T}f(s,{X}_{s,}Y_{s},{Z}_{s})ds-\int_{t}^{T}{Z}_{s}^{\intercal}d{W}_{s}\\
{X}_{t} &={x}+\int_{0}^{t}{\alpha}(s,{X}_{s})ds+\int_{0}^{t}{\beta}(t,{X})d{W}_{s}
\end{align*}
where
$$
f(t,{X},Y,{Z})	=(1-\gamma)r(t,{X})+\frac{1-\gamma}{2\gamma}\parallel{\theta}(t,{X})\parallel^{2}-\delta\vartheta+\frac{\vartheta}{\psi}\delta^{\psi}e^{-\frac{\psi}{\vartheta}Y}+\frac{1-\gamma}{\gamma}{\theta}^{\intercal}{Z}+\frac{1}{2\gamma}\parallel{Z}\parallel^{2}
$$


## DNN algorithm

Consider a partition of the time interval $[0,T]$, $\mathcal{T}$:
$0=t_{0}\leq t_{1}\leq\ldots\leq t_{N}=T$, with $\Delta t=T/N$ and
$t_{i}=i\Delta t$, for all $i=0,1,\ldots,N$, and $\Delta{W}_{i}={W}_{t_{i+1}}-{W}_{t_{i}}$,
for all $i=0,1,\ldots,N-1$. 

Inspired by the nonlinear Feynman-Kac
formula, we can view
${Z}_{t}$ as a function of ${X}_{t}$ and $Y_{t}$. Then our
goal becomes finding approximation functions
$\phi_{i}:\mathbb{R}^{m}\times\mathbb{R}\rightarrow\mathbb{R}^{d}$
for $i=0,1,\ldots,N-1$ such that $\phi_{i}(\hat{{X}}_{t_{i}},\hat{Y}_{t_{i}})$
can serve as good approximation of  $Z_{t_{i}}$. 

The initial value of $Y$ can be set as another training variable $\mu_0$ which will later be determined by gradient decsent.

With the DNN functions $\phi_{i}(0\leq i\leq N-1)$ and the training variable $\mu_0$, we can perform the Monte Carlo simulation based on the Euler scheme
$$
\begin{cases}
\hat{{X}}_{0}={x},\quad\hat{Y}_{0}=\mu_{0},\\
\hat{{X}}_{t_{i+1}}=\hat{{X}}_{t_{i}}+{\alpha}(t_{i},{X}_{t_{i}})\Delta t+{\beta}(t_{i},\hat{{X}}_{t_{i}})\Delta{W}_{i}\\
\hat{{Z}}_{t_{i}}=\phi_{i}(\hat{{X}}_{t_{i}},\hat{Y}_{t_{i}})\\
\hat{Y}_{t_{i+1}}=\hat{Y}_{t_{i}}-f(t_{i},\hat{{X}}_{t_{i}},\hat{Y}_{t_{i}},\hat{{Z}}_{t_{i}})\Delta t+\hat{{Z}}_{t_{i}}\Delta{W}_{i}
\end{cases}
$$
To determine $\mu_{0}$ and $\phi_{i}(0\leq i\leq N-1)$, we solve the following stochastic optimization problem
$$
\inf_{\mu_{0},\phi_{i}\in\mathcal{N}(\Phi_{i})}J(\mu_0,\Phi_{0},\ldots,\Phi_{N-1}):=\mathbb{E}\vert \eta-\hat{Y}_{T}\vert^{2}
$$
where $\mathcal{N}(\Phi)$ denotes the set of neural networks with paremeters summarized in the set $\Phi$

# Stochastic volatility model

In this example, the finiancial market has only one risky assets governed by the Heston stochastic Heston model. This is a one dimesional problem with a closed form solution under some specific settings in terms of the value of Risk Aversion and EIS. We can compare the numerical result with the closed form solution to test the accuracy of the DNN method.

$$dS_{t}^{1}=S_{t}[(r+\bar{\mu}X_{t})dt+\sqrt{X_{t}}dW_{t}^{1}]$$
$$dX_{t}=\kappa(\bar{X}-X_{t})dt+\bar{\beta}\sqrt{X_{t}}(\rho dW_{t}^{1}+\sqrt{1-\rho^{2}}dW_{t}^{2})$$
$$\kappa = 5,\quad \bar{X}=0.15^2, \quad \bar{\mu}=3.11, \quad \bar{\beta}=0.25,\quad \rho = -0.5$$

In [2]:
problem_name = "Heston"
config = Config(horizon=2, steps=20, gamma=2, psi=0.125, x0=0.25)
equation = get_equation(problem_name, config)

* `horizon`: The investment horizon $T$
* `steps`: the number of time steps in one year. $n$, i.e. $\Delta t = 1/n$
* `gamma`: Risk Aversion
* `psi`: EIS
* `x0` : the inital value of the state variable, which is the variance here.

## Generate trainig data

* `equation`: define the FBSDE related Heston Model
* `n_epoch`: how many times the training procesure repeats
* `batch_size`: how many traing data in one epoch of training
* `test_size`: number of test data for one epoch
To better control the comparison with other numerical method, I generate all the Brownian motion first using a fixed seed, and treat the Brownian motions as the *input training data* as that in machine learning context.

In [3]:
batch_size = 512
test_size = 1280
n_epochs = 20
total_sample_size = batch_size * n_epochs + test_size
test_size_ratio = test_size / total_sample_size
np.random.seed(1)
input_data = np.random.normal(0, equation.sqrt_delta_t, (total_sample_size, equation.dim * equation.num_time_interval))
outcome_data = np.ones([total_sample_size, 1])
X_train, X_test, y_train, y_test = train_test_split( input_data, outcome_data, test_size=test_size_ratio, random_state=42)
train_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(X_test).batch(test_size)

## Train model

In [4]:
model = dBSDE(equation, y0=1.0)
history = model.custom_fit(train_ds=train_ds, test_ds=test_ds, epochs=n_epochs)

Epoch: 1 Elapsed time:  10.29696774482727 y0:  1.1577301 z0:  -0.0234327409 Test loss:  0.000877402141 lr:  0.01
Epoch: 2 Elapsed time:  11.271549701690674 y0:  1.1379776 z0:  -0.0436525941 Test loss:  1.82423973e-05 lr:  0.01
Epoch: 3 Elapsed time:  12.23916506767273 y0:  1.13607848 z0:  -0.0539336205 Test loss:  4.67429209e-06 lr:  0.01
Epoch: 4 Elapsed time:  13.211825847625732 y0:  1.13809538 z0:  -0.0566247106 Test loss:  9.32364742e-07 lr:  0.01
Epoch: 5 Elapsed time:  14.18775987625122 y0:  1.13756526 z0:  -0.0563529693 Test loss:  2.71212855e-07 lr:  0.01
Epoch: 6 Elapsed time:  15.166388750076294 y0:  1.1374805 z0:  -0.0558502041 Test loss:  2.16808345e-07 lr:  0.01
Epoch: 7 Elapsed time:  16.133893728256226 y0:  1.1376555 z0:  -0.0557476021 Test loss:  1.97283697e-07 lr:  0.01
Epoch: 8 Elapsed time:  17.108829498291016 y0:  1.13755035 z0:  -0.0557993613 Test loss:  1.83004232e-07 lr:  0.01
Epoch: 9 Elapsed time:  18.086017847061157 y0:  1.13758564 z0:  -0.0558170043 Test loss

## Exact solution

In [5]:
def exact_solution(equation):
    if equation.psi == 2-equation.gamma+(equation.rho*(1-equation.gamma))**2/equation.gamma:
        Y_exact = equation.h_exact(config.x_init, 0, config.total_time)
        Z_exact = equation.beta_bar * np.sqrt(config.x_init) * equation.hx_exact(config.x_init, 0, config.total_time)
        print(f"Y0_true: {Y_exact:.4f}, Z0_true: {Z_exact:.4f}")
    else:
        print('No exact solution')
exact_solution(equation)

Y0_true: 1.1420, Z0_true: -0.0554


The exact solution is $Y_0=1.142,Z_0=-0.0554$, the numerical result given by DNN is $\hat{Y}_0=1.138, \hat{Z_0}=-0.0558$.

# Large scale model

In the financial market, There are $d$ Brownian motions and $d+1$ securities, $d-1$ of which are stock portfolios, one mutual fund of long term bounds and one riskless asset. There are $2d$ state variables: 
 $d$ marketprices of risk, one interest rate and  $d-1$ dividends affecting the evolution of the market prices of risks.

The dynamics of the interest rate $r$ and the price of the bond portfolio ($S_{b}$) are 
\begin{align}
dr_{t} & =\kappa_{r}(\bar{r}-r_{t})(1+\phi_{r}(\bar{r}-r_{t})^{2\eta_{r}})dt-\sigma_{r}r_{t}^{\gamma_{r}}dW_{1t}\nonumber \\
dS_{bt} & =S_{bt}\Big[(r_{t}+\sigma_{b}\theta_{1t})dt+\sigma_{b}dW_{1t}\Big]\label{eq:sb}
\end{align}
The first stock portfolio is the market portfolio of securities, which
depends on $(W_{1},W_{2})$
\begin{align}
dS_{1t}+S_{1t}p_{2t}dt & =S_{1t}\Big(r_{t}+\sigma_{1}(\rho_{11}\theta_{1t}+\sqrt{1-\rho_{11}^{2}}\theta_{2,t})\Big)dt\label{eq:s1}\\
 & \quad+S_{1t}\sigma_{1}\Big(\rho_{11}dW_{1t}+\sqrt{1-\rho_{11}^{2}}dW_{2t}\Big)\nonumber 
\end{align}
The next $d-2$ funds are independent with each other as well as the
market portfolio and the bond fund, i.e. the fund $j$ is perfectly
correlated with $W_{j+1}$, $\rho_{jk}=0$, for $j=2,\ldots,d-1$.
They are pure hedge funds. The dynamics of these $d-2$ funds are
as shown as follws
\begin{equation}
dS_{it}+S_{it}p_{i+1,t}dt=S_{it}\Big(r_{t}+\sigma_{i}\theta_{i+1,t}\Big)dt+S_{it}\sigma_{i}dW_{i+1,t},\ i=2,\cdots,d-1\label{eq:si}
\end{equation}



The state variables are 
$$ \big(r,p_{2},\ldots,p_{d},\theta_{1},\theta_{2}\ldots,\theta_{d}\big)_{2d} $$
These $2d$ state variables all have their own dynamics. For 
The market price of the bond portfolio satisfies the following process
without dividend effect.
\begin{align*}
d\theta_{1t} & =\big(\kappa_{\theta_{1}}(\bar{\theta}_{1}-\theta_{1t})+\mu_{\theta_{1}}^{r}(r_{t},\theta_{1t})\big)dt+\sigma_{\theta_{1}}(\theta_{1t})dW_{1t}\\
\sigma_{\theta_{1}}(\theta_{1t}) & =\tilde{\sigma}_{\theta_{1}}(\theta_{1l}+\theta_{1t})^{\gamma_{1\theta_{1}}}(1-(\frac{\theta_{1l}+\theta_{1t}}{\theta_{1l}+\theta_{1u}})^{1-\gamma_{1\theta_{1}}})^{\gamma_{2\theta_{1}}}\\
\mu_{\theta_{1}}^{r}(r_{t},\theta_{1t}) & =\delta_{r1}(\bar{r}-r_{t})(\theta_{1l}+\theta_{1t})(1-\frac{\theta_{1l}+\theta_{1t}}{\theta_{1l}+\theta_{1u}})
\end{align*}

For $i=2,\ldots,d$, the pair $(\theta_{i},p_{i})$ satisfies identical
process as follows:
\begin{align*}
d\theta_{it} & =\big(\kappa(\bar{\theta}-\theta_{it})+\mu_{\theta}^{r}(r_{t},\theta_{it})+\mu_{\theta}^{p}(p_{t},\theta_{it})\big)dt+\sigma_{\theta}(\theta_{it})dW_{it}\\
dp_{it} & =\kappa_{p}(\bar{p}-p_{it})(1+\phi_{p}(\bar{p}-p_{it})^{2\eta_{p}})dt-\sigma_{p}p_{it}^{\gamma_{p}}dW_{it}
\end{align*}
\begin{align*}
\mu_{\theta}^{r}(r_{t},\theta_{it}) & =\delta_{r\theta}(\bar{r}-r_{t})(\theta_{l}+\theta_{it})(1-\frac{\theta_{l}+\theta_{it}}{\theta_{l}+\theta_{u}})\\
\mu_{\theta}^{p}(p_{t},\theta_{it}) & =\delta_{p\theta}(\bar{p}-p_{t})(\theta_{l}+\theta_{it})(1-\frac{\theta_{l}+\theta_{it}}{\theta_{l}+\theta_{u}})\\
\sigma_{\theta}(\theta_{it}) & =\tilde{\sigma}_{\theta}(\theta_{l}+\theta_{it})^{\gamma_{1\theta}}(1-(\frac{\theta_{l}+\theta_{it}}{\theta_{l}+\theta_{u}})^{1-\gamma_{1\theta}})^{\gamma_{2\theta}}
\end{align*}

In [5]:
problem_name = "LargeScale"
config = Config(d=100, horizon=2, steps=20, gamma=4, psi=1.2, x0=0.5)
equation = get_equation(problem_name, config)

Let's take dimesion $d=100$ as an example. We choose the investment horizon to be 2 years, the Risk Aversion to be 4 and the EIS to be 1.2. The training time of 20 epochs is around 90s

In [6]:
batch_size = 512
test_size = 1280
n_epochs = 20
total_sample_size = batch_size * n_epochs + test_size
test_size_ratio = test_size / total_sample_size
np.random.seed(1)
input_data = np.random.normal(0, equation.sqrt_delta_t, (total_sample_size, equation.dim * equation.num_time_interval))
outcome_data = np.ones([total_sample_size, 1])
X_train, X_test, y_train, y_test = train_test_split( input_data, outcome_data, test_size=test_size_ratio, random_state=42)
train_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices(X_test).batch(test_size)

In [7]:
model = dBSDE(equation, y0=1.0)
history = model.custom_fit(train_ds=train_ds, test_ds=test_ds, epochs=n_epochs)

Epoch: 1 Elapsed time:  23.073079347610474 y0:  1.18148184 z0:  [[0.00732259732 0.0015855619 0.00325800665 ... -0.00238456856 -0.000908814487 0.00350722275]] Test loss:  0.00149167969 lr:  0.01
Epoch: 2 Elapsed time:  26.654821634292603 y0:  1.24709189 z0:  [[0.00667434372 0.000969646382 0.00256709731 ... -0.00166841806 -0.000294668949 0.00061363104]] Test loss:  0.00047707255 lr:  0.01
Epoch: 3 Elapsed time:  30.24810266494751 y0:  1.22331476 z0:  [[0.00297889323 -0.000526351447 0.000604598958 ... 0.000252651138 8.22625152e-05 -0.000591267541]] Test loss:  9.56056647e-06 lr:  0.01
Epoch: 4 Elapsed time:  33.85690355300903 y0:  1.2203021 z0:  [[0.00299151568 -0.000182131655 -5.2196192e-05 ... 0.00032105908 2.53057096e-05 -0.000160081574]] Test loss:  1.00158541e-05 lr:  0.01
Epoch: 5 Elapsed time:  37.4552788734436 y0:  1.22445989 z0:  [[0.00368735823 8.5366e-05 -0.000105258769 ... -3.55265511e-05 -7.44397476e-05 0.00012440898]] Test loss:  3.07005462e-06 lr:  0.01
Epoch: 6 Elapsed tim